In [4]:
#imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient

%matplotlib inline

In [5]:
#read in api
load_dotenv()
api_key = os.getenv('NEWS_API_KEY')

In [6]:
#newsapi client
newsapi = NewsApiClient(api_key = api_key)

In [15]:
#bitcoin headlines
btc_headlines = newsapi.get_everything(
    q = 'bitcoin',
    language = 'en',
    sort_by = 'relevancy'
)

In [16]:
#ethereum headlines
eth_headlines = newsapi.get_everything(
    q = 'ethereum',
    language = 'en',
    sort_by = 'relevancy'
)

In [17]:
sentiments = []

for articles in btc_headlines['articles']:
    try:
        text = articles['content']
        results = analyzer.polarity_scores(text)
        compound = results['compound']
        pos = results['pos']
        neu = results['neu']
        neg = results['neg']
        
        sentiments.append({
            'Compound': compound,
            'Positive': pos,
            'Negative': neg,
            'Neutral': neu,
            'text': text,
        })
    except AttributeError:
        pass
    
btc = pd.DataFrame(sentiments)
btc.head()

,Compound,Positive,Negative,Neutral,text
0,0.3612,0.077,0.000,0.923,Musk.MARK RALSTON/AFP via Getty Images\r\nElon...
1,0.0000,0.000,0.000,1.000,When Denis Rusinovich set up cryptocurrency mi...
2,0.3182,0.105,0.000,0.895,El Salvador introduced Bitcoin as a legal tend...
3,-0.4404,0.000,0.083,0.917,Were officially building an open Bitcoin minin...
4,-0.3182,0.045,0.084,0.871,Israeli national Tal Prihar pled guilty to rou...


In [18]:
sentiments = []

for articles in eth_headlines['articles']:
    try:
        text = articles['content']
        results = analyzer.polarity_scores(text)
        compound = results['compound']
        pos = results['pos']
        neu = results['neu']
        neg = results['neg']
        
        sentiments.append({
            'Compound': compound,
            'Positive': pos,
            'Negative': neg,
            'Neutral': neu,
            'text': text,
        })
    except AttributeError:
        pass
    
eth = pd.DataFrame(sentiments)
eth.head()

,Compound,Positive,Negative,Neutral,text
0,0.0000,0.000,0.000,1.000,"In a new blog post the company said that 4,836..."
1,0.0000,0.000,0.000,1.000,Hackers who made off with roughly $15 million ...
2,0.1779,0.052,0.000,0.948,"On some level, the new mayor is simply employi..."
3,0.0772,0.038,0.000,0.962,"Back in September\r\n, Robinhood announced pla..."
4,-0.1027,0.056,0.067,0.877,Trading platform Crypto.com lost about $34 mil...


In [23]:
#summary statistics btc
btc.describe()

,Compound,Positive,Negative,Neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.041505,0.058200,0.037900,0.903900
std,0.366363,0.057018,0.043444,0.062064
min,-0.440400,0.000000,0.000000,0.765000
25%,-0.331550,0.000000,0.000000,0.875500
50%,0.038600,0.054000,0.000000,0.917000
75%,0.328950,0.084000,0.083250,0.934000
max,0.680800,0.185000,0.101000,1.000000


In [21]:
#summary statistics eth
eth.describe()

,Compound,Positive,Negative,Neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.164365,0.053050,0.008700,0.938200
std,0.272452,0.065873,0.021502,0.066563
min,-0.153100,0.000000,0.000000,0.783000
25%,0.000000,0.000000,0.000000,0.894000
50%,0.000000,0.039500,0.000000,0.951000
75%,0.406800,0.090250,0.000000,1.000000
max,0.757900,0.217000,0.067000,1.000000
